## Full data

Generemos acá los datos con las etiquetas


Carguemos primero las etiquetas...

In [37]:
import glob 
import pandas as pd
from pandarallel import pandarallel

pandarallel.initialize()

labels = pd.concat([pd.read_csv(path) for path in glob.glob("../../data/predictions/*.csv")])
del labels["Unnamed: 0"]

labels.set_index("id", inplace=True)


INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


Tenemos dos problemas:

1. Los datos que nos quedaron sin contexto
2. Los que no clasificamos :-) (son los últimos!)

In [38]:
import ijson
from tqdm.auto import tqdm

path = "../../data/news-all-final.json"
num_comments = 0
num_articles = 537_000
data = []

not_classified = []
without_context = []

with open(path) as f:
    articles = ijson.items(f, 'item')
    for article in tqdm(articles, total=num_articles):
        num_comments += len(article["comments"])

        context = article["title"] if "title" in article else article["text"]

        for comment in article["comments"]:
            datum = {
                "user": article["user"],
                "text": comment["text"],
                "context": context,
                "article": article,
                "id": comment["tweet_id"],
            }

            data.append(datum)
            if datum["id"] not in labels.index:
                not_classified.append(datum)
            elif len(datum["context"]) <= 2:
                without_context.append(datum)

    print(num_comments, " comments processed")


8569648  comments processed


In [39]:
len(without_context), len(not_classified)

(16854, 76481)

In [40]:
import fire
import os
import torch
import ijson
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from hatedetection.training import tokenize
from hatedetection.preprocessing import preprocess_tweet
from torch.utils.data import DataLoader
from datasets import Dataset, Value, ClassLabel, Features
from hatedetection import extended_hate_categories
from pandarallel import pandarallel
from hatedetection import HateSpeechAnalyzer

def get_outputs(model, dataloader):
    outputs = []

    for batch in tqdm(dataloader):
        batch = {k:v.to(device) for k, v in batch.items() if k != "id"}
        outs = model(**batch)
        outputs.append((outs.logits > 0).cpu().numpy())

    return np.vstack(outputs)



def build_dataset(df, tokenizer, batch_size):
    features = Features({
        'id': Value('uint64'),
        'context': Value('string'),
        'text': Value('string'),
    })


    dataset = Dataset.from_pandas(df, features=features)

    dataset = dataset.map(
        lambda x: tokenizer(x["text"], x["context"], padding="longest", truncation='longest_first'),
        batched=True, batch_size=batch_size,
    )
    def format_dataset(dataset):
        dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask'])
        return dataset

    dataset = format_dataset(dataset)

    return dataset

def process_comments(model, tokenizer, data, batch_size):
    df = pd.DataFrame(data)

    df["text"] = df["text"].parallel_apply(preprocess_tweet)
    df["context"] = df["context"].parallel_apply(preprocess_tweet)
    batch_size = 16

    dataset = build_dataset(df, tokenizer, batch_size)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    outputs = get_outputs(model, dataloader)


    df[extended_hate_categories] = outputs
    df["HATEFUL"] = df[extended_hate_categories[1:]].sum(axis=1) > 0

    return df


In [41]:
from hatedetection import HateSpeechAnalyzer

analyzer = HateSpeechAnalyzer.load_contextualized_model()

device = "cuda"

model = analyzer.base_model
model = model.to(device)
model.eval()

tokenizer = analyzer.tokenizer

Perfecto, tenemos ~80K para clasificar de nuevo

In [48]:
to_classify = []

for comment in without_context + not_classified:
    to_classify.append({
        "text": comment["text"],
        "context": comment["article"]["text"],
        "id": comment["id"]
    })


In [ ]:
df_others = process_comments(model, tokenizer, to_classify, 16)
df_others.set_index("id", inplace=True)

Veamos que los primeros estén todos (ya los clasificamos!) y que el resto no esté

In [69]:
all(i in labels.index for i in df_others.index[:len(without_context)])

True

In [64]:
all(i not in df.index for i in df_others.index[len(without_context):])

True

Bien. Entonces, la primera parte la tenemos que actualizar

In [71]:
len_without = len(without_context)

labels.loc[df_others.index[:len_without]] = df_others[:len_without][extended_hate_categories]

...y la segunda la concatenamos

In [75]:
labels = pd.concat([labels, df_others[extended_hate_categories].iloc[len_without:]])

In [76]:
len(labels)

8569648

BIEN CARAJO!

In [80]:
article["comments"][:5]

[{'tweet_id': 1227965992468389888,
  'text': '@elmundoes Estrasburgo es fascista',
  'user_id': 1121326270011322369,
  'created_at': {'$date': '2020-02-13T14:41:21Z'},
  'hateful_value': Decimal('0.10774408280849457')},
 {'tweet_id': 1227966561610280960,
  'text': '@elmundoes Pues eso, cuando el PSOE dice algo ocurre lo contrario',
  'user_id': 1115886910004781056,
  'created_at': {'$date': '2020-02-13T14:43:37Z'},
  'hateful_value': Decimal('0.00374859431758523')},
 {'tweet_id': 1227967028855787522,
  'text': '@elmundoes Haber que dicen ahora los progres izquierdistas que tanto han criticado.\nAhora que no pase ni uno solo, que entre la frontera ilegalmente',
  'user_id': 2407765990,
  'created_at': {'$date': '2020-02-13T14:45:28Z'},
  'hateful_value': Decimal('0.6550083160400391')},
 {'tweet_id': 1227967240676487171,
  'text': '@elmundoes Jodeeer...había leído "Ceuta y Manila" https://t.co/CiD2pwEZtI',
  'user_id': 361590424,
  'created_at': {'$date': '2020-02-13T14:46:19Z'},
  'hate

In [118]:
labels.to_csv("labels.csv")

In [119]:
import json
def preprocess_article(article):
    ret = {k:v for k, v in article.items() if k not in {"_id", "slug"}}
    ret["created_at"] = ret["created_at"]["$date"]
    ret["tweet_id"] = str(ret["tweet_id"])
    for comment in ret["comments"]:
        comment["created_at"] = comment["created_at"]["$date"]
        if "hateful_value" in comment:
            del comment["hateful_value"]

        comment["user_id"] = str(comment["user_id"])
        comment["prediction"] = dict((1* labels.loc[comment["tweet_id"]]).astype(int))
        comment["prediction"] = {k:int(v) for k, v in comment["prediction"].items()}
        comment["tweet_id"] = str(comment["tweet_id"])
    return ret

def save_articles_to(articles, path):
    with open(path, "w+") as f:
        json.dump(articles, f)
    print(f"Salvados {len(articles)} artículos en {path}")


processed_articles = []
BATCH_SIZE = 50_000
batch_num = 1

base_path = "../../data/news/"

with open("../../data/news-all-final.json") as f:
    articles = ijson.items(f, 'item')
    for article in tqdm(articles, total=num_articles):
        article = preprocess_article(article)

        processed_articles.append(article)

        if len(processed_articles) > BATCH_SIZE:
            path = os.path.join(base_path, f"news-{batch_num}.json")
            save_articles_to(processed_articles, path)
            processed_articles = []
            batch_num += 1

if len(processed_articles):
    path = os.path.join(base_path, f"news-{batch_num}.json")
    save_articles_to(processed_articles, path)
            


Salvados 50001 artículos en ../../data/news/news-1.json
Salvados 50001 artículos en ../../data/news/news-2.json
Salvados 50001 artículos en ../../data/news/news-3.json
Salvados 50001 artículos en ../../data/news/news-4.json
Salvados 50001 artículos en ../../data/news/news-5.json
Salvados 50001 artículos en ../../data/news/news-6.json
Salvados 50001 artículos en ../../data/news/news-7.json
Salvados 50001 artículos en ../../data/news/news-8.json
Salvados 50001 artículos en ../../data/news/news-9.json
Salvados 50001 artículos en ../../data/news/news-10.json

Salvados 37191 artículos en ../../data/news/news-11.json


In [ ]:
df_comments = pd.DataFrame(data)
df_comments.set_index("id", inplace=True)

In [16]:
without_context = []
for comment in tqdm(data):
    if len(comment["context"]) <= 2:
        without_context.append(comment)

len(without_context)


16948

In [15]:
pd.DataFrame(without_context)["user"].value_counts()

LaVanguardia       4318
elmundoes          4002
infobae            2488
clarincom          1720
LANACION           1404
latercera           778
laderechamedios     549
pagina12            521
elpaisuy            423
laderechadiario     305
perfilcom           274
cronica              88
izquierdadiario      77
abc_es                1
Name: user, dtype: int64

In [17]:
import glob 
import pandas as pd

df_comments = pd.DataFrame(data)
labels = pd.concat([pd.read_csv(path) for path in glob.glob("../../data/predictions/*.csv")])
del labels["Unnamed: 0"]

df_comments.set_index("id", inplace=True)
labels.set_index("id", inplace=True)


In [18]:
from hatedetection import extended_hate_categories

df = labels.merge(df_comments, left_index=True, right_index=True)
df = df.reindex(columns=[ 'user', 'text', 'context'] + extended_hate_categories)

In [24]:

def preprocess_article(article):
    ret = {k:v for k, v in article.items() if k not in {"_id", "slug"}}
    ret["created_at"] = ret["created_at"]["$date"]
    return ret


with open(path) as f:
    articles = ijson.items(f, 'item')
    for article in tqdm(articles, total=num_articles):
        article = preprocess_article(article)
        for comment in article["comments"]:
            df.loc[comment["tweet_id"]]


KeyboardInterrupt: 

In [4]:
df["HATEFUL"] = (df[extended_hate_categories[1:]].sum(axis=1) > 0)

df.shape, df[extended_hate_categories].sum()

((8493167, 13),
 CALLS          29632
 WOMEN          61599
 LGBTI          24969
 RACISM         56714
 CLASS          27234
 POLITICS      120153
 DISABLED       57296
 APPEARANCE    130832
 CRIMINAL       46544
 dtype: int64)

In [5]:
labels.shape

(8493167, 9)

In [87]:
df["user"].value_counts()

elmundoes          215256
clarincom          204698
infobae            192993
LANACION           168549
abc_es             124447
latercera           94087
elpaisuy            36911
LaVanguardia        32197
cronica             24265
perfilcom           17759
laderechadiario      9788
pagina12               45
izquierdadiario        19
Name: user, dtype: int64

In [90]:
df[df["user"] == "elmundoes"][extended_hate_categories].sum()

CALLS          317
WOMEN         1293
LGBTI          419
RACISM        1124
CLASS          185
POLITICS      1138
DISABLED       796
APPEARANCE     935
CRIMINAL       293
dtype: int64

In [92]:
pd.options.display.max_colwidth = 250
pd.options.display.max_rows = 500
pd.options.display.min_rows = 300
df[ df["CLASS"] & (df["user"] == "elmundoes")][["text", "context"]]

,text,context
id,,
1257616051543642112,@elmundoes Que se vayan ellos los parásitos y nos ahorramos además de dinero vagos,Vox propone en el Congreso eliminar la mitad de los ministerios como medida de ahorro ante la crisis
1257622410758676481,@elmundoes Muy buena idea ya q la mayoria de ellos no hacen ni una puta mierda vagos de mierda y lo q se quedan se le rebaja el sueldo,Vox propone en el Congreso eliminar la mitad de los ministerios como medida de ahorro ante la crisis
1257700499970838530,"@elmundoes Sindicatos, que cojones es eso\nCerdos que se engordan de chutes,putas y carabineros\nSindicatos a la puta mierda",¿Qué pasará con los más de tres millones de afectados por ERTE si decae el estado de alarma sin un nuevo acuerdo con patronal y sindicatos?
1258096696359292930,"@elmundoes Y van a dar una renta mínima vital para siempre ? En fin , creando vagos. Eso se puede hacer durante un periodo limitado","La factura por ERTE, desempleo y ayudas a empresas asciende a 11.000 millones desde marzo"
1258125680283451393,"@elmundoes Una Auténtica vergüenza. Este gobierno pagando a vagos, okupas, inmigrantes ilegales, menas.... Y mientras los trabajadores esenciales medicos, policias etc... Con sueldos míseros jugandose sus vidas y las de sus familias. No tienen pe...","La nómina de un médico contratado en plena pandemia: 1.046,58 euros"
1258772304647065601,@elmundoes A seguir alimentando a los vagos,España se alía con Italia y Portugal para proponer una renta mínima europea
1258786865064357888,"@elmundoes @rojas1977 Los pachachos de la chorprecha de Newtral y la Ser.\nMorcillas de Burgos de parte de Telefónica, Deepak Daswani y Movistar +.\nCero disculpas hasta el momento. https://t.co/6tKz2vpQDs",La derrota de la esvástica: la posibilidad de un nuevo Hitler
1259766910746583041,@elmundoes Paguita destinada a los okupas y a etnianos,"El perfil para cobrar el ingreso mínimo vital: mayor de 23 años, sin ingresos y sin vivienda en propiedad"
1260339740714446851,"@elmundoes @congurb Ya está allanado el camino para que toda la patulea de vagos que no les sale de los cojones trabajar cobren su paguita., nos está quedando una España cojonuda...",Declarar que no se tienen ingresos bastará para cobrar la renta mínima vital


In [75]:
df.to_csv("comentarios_analizados.csv")